## Hello!

I'm AmirHossein Rezaaie with Student-Number 403541474 ; 

Project Titlle : Data Structures Project  

In [10]:
strlist = input().replace(" ", "").replace("\t", "").replace("\n", "")
print(f"Input String: {strlist}")


Input String: (2+3)*4^2+√16


In [11]:
normal_input = ""
i = 0
while i < len(strlist):
    current = strlist[i]

    if current in '0123456789()*/^√':
        normal_input += current
        i += 1

    elif current in '+-':
        prev_char = strlist[i-1] if i > 0 else ''
        if prev_char in '0123456789)':
            normal_input += current
            i += 1
        else:
            count_minus = 0
            while i < len(strlist) and strlist[i] in '+-':
                if strlist[i] == '-':
                    count_minus += 1
                i += 1
            if count_minus % 2 == 1:
                normal_input += '-'
                
    else:
        normal_input += current
        i += 1

print(f"Normal Input: {normal_input}")

Normal Input: (2+3)*4^2+√16


In [12]:
class Token:
    def __init__(self, type_, value):
        self.type = type_
        self.value = value

    def __repr__(self):
        return f"Token({self.type}, {repr(self.value)})"

    def __str__(self):
        return f"{self.type}: {self.value}"

tokens = []
i = 0
while i < len(normal_input):
    ch = normal_input[i]

    # عدد
    if ch in "0123456789":
        num = 0
        while i < len(normal_input) and normal_input[i] in "0123456789":
            num = num * 10 + int(normal_input[i])
            i += 1
        tokens.append(Token("NUMBER", num))
        continue

    # عملگرها
    elif ch in "+-*/^":
        tokens.append(Token("OPERATOR", ch))
        i += 1

    # رادیکال
    elif ch == "√":
        tokens.append(Token("OPERATOR", "√"))
        i += 1

    # پرانتزها
    elif ch == "(":
        tokens.append(Token("LPAREN", "("))
        i += 1
    elif ch == ")":
        tokens.append(Token("RPAREN", ")"))
        i += 1

    else:
        print(f"کاراکتر ناشناخته: {ch}")
        i += 1

print(f"Tokens : {tokens}")

Tokens : [Token(LPAREN, '('), Token(NUMBER, 2), Token(OPERATOR, '+'), Token(NUMBER, 3), Token(RPAREN, ')'), Token(OPERATOR, '*'), Token(NUMBER, 4), Token(OPERATOR, '^'), Token(NUMBER, 2), Token(OPERATOR, '+'), Token(OPERATOR, '√'), Token(NUMBER, 16)]


In [14]:
def precedence(op):
    if op in ("+", "-"):
        return 1
    if op in ("*", "/"):
        return 2
    if op == "^":
        return 3
    if op == "√":
        return 4
    return 0

stack = []
output_list = []

for token in tokens:
    if token.type == "LPAREN":
        stack.append(token.value)
        
    elif token.type == "RPAREN":
        while stack and stack[-1] != "(":
            output_list.append(stack.pop())
        if stack:
            stack.pop()
            
    elif token.type == "NUMBER":
        output_list.append(token.value)
        
    elif token.type == "OPERATOR":
        current_op = token.value
        while (stack and 
            stack[-1] != "(" and 
            precedence(stack[-1]) >= precedence(current_op) and
            (current_op in ("+", "-", "*", "/", "^", "√"))):
            output_list.append(stack.pop())
        stack.append(current_op)

while stack:
    output_list.append(stack.pop())

print(f"Output Lise is: {output_list}")

Output Lise is: [2, 3, '+', 4, 2, '^', '*', 16, '√', '+']
The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


In [ ]:
class Node:
    def __init__(self, value):
        self.value = value
        self.left = None
        self.right = None


def build_expression_tree(postfix):
    stack = []
    
    for token in postfix:
        if isinstance(token, int):
            stack.append(Node(token))
        else:
            node = Node(token)
            
            if token == "√":
                if not stack:
                    raise ValueError("Invalid expression: missing operand for √")
                node.left = stack.pop()
                node.right = None
            else:
                if len(stack) < 2:
                    raise ValueError("Invalid expression: not enough operands")
                right = stack.pop()
                left = stack.pop()
                node.left = left
                node.right = right
            
            stack.append(node)
    
    if len(stack) != 1:
        raise ValueError("Invalid expression: multiple roots remain")
    
    return stack[0]

In [ ]:
root = build_expression_tree(output_list)

def print_tree(node, level=0, prefix="Root: "):
    if node:
        print("  " * level + prefix + str(node.value))
        print_tree(node.left, level + 1, "L-- ")
        print_tree(node.right, level + 1, "R-- ")

print_tree(root)

In [ ]:
import math

def evaluate_tree(node):
    if node is None:
        return 0
    
    if isinstance(node.value, int):
        return node.value

    if node.value == "+":
        return evaluate_tree(node.left) + evaluate_tree(node.right)
    if node.value == "-":
        return evaluate_tree(node.left) - evaluate_tree(node.right)
    if node.value == "*":
        return evaluate_tree(node.left) * evaluate_tree(node.right)
    if node.value == "/":
        right_val = evaluate_tree(node.right)
        if right_val == 0:
            raise ZeroDivisionError("Division by zero occurred!")
        return evaluate_tree(node.left) / right_val
    if node.value == "^":
        return math.pow(evaluate_tree(node.left), evaluate_tree(node.right))
    if node.value == "√":
        val = evaluate_tree(node.left)
        if val < 0:
            raise ValueError("Cannot take square root of negative number")
        return math.sqrt(val)
    
    raise ValueError(f"Unknown operator: {node.value}")

In [ ]:
try:
    root = build_expression_tree(output_list)
    print("Expression tree built successfully!")
    
    result = evaluate_tree(root)
    print(f"Final result = {result}")

except ZeroDivisionError as e:
    print(f"Error: {e}")
except Exception as e:
    print(f"Syntax or calculation error: {e}")